In [1]:
#import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
gpd.options.use_pygeos = False
import seaborn as sns
import os
import numpy as np

In [2]:
from shapely.geometry import Point, Polygon
import shapely
shapely.speedups.disable()

In [4]:
nb = os.path.join('Resources',"Neighbourhoods","Neighbourhoods.shp")
regions = gpd.read_file(nb)
regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()
regions.head()

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_14,FIELD_15,geometry,neighbourhood
0,2101,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680...",wychwood
1,2102,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704...",yonge-eglinton
2,2103,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680...",yonge-st.clair
3,2104,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((-79.50529 43.75987, -79.50488 43.759...",york university heights
4,2105,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((-79.43969 43.70561, -79.44011 43.705...",yorkdale-glen park


In [5]:
regions['geometry']

0      POLYGON ((-79.43592 43.68015, -79.43492 43.680...
1      POLYGON ((-79.41096 43.70408, -79.40962 43.704...
2      POLYGON ((-79.39119 43.68108, -79.39141 43.680...
3      POLYGON ((-79.50529 43.75987, -79.50488 43.759...
4      POLYGON ((-79.43969 43.70561, -79.44011 43.705...
                             ...                        
135    POLYGON ((-79.24549 43.73060, -79.24555 43.730...
136    POLYGON ((-79.40401 43.64719, -79.40419 43.647...
137    POLYGON ((-79.55236 43.70947, -79.55229 43.709...
138    POLYGON ((-79.51703 43.64611, -79.51717 43.646...
139    POLYGON ((-79.28857 43.79607, -79.28921 43.795...
Name: geometry, Length: 140, dtype: geometry

In [6]:
#build dataframe
lights_df = pd.read_csv('Resources/Red Light Cameras Data.csv')
lights_df.head()

,_id,INTERSECTION_ID,LINEAR_NAME_FULL_1,LINEAR_NAME_FULL_2,ID,X,Y,LONGITUDE,LATITUDE,OBJECTID,...,ADDITIONAL_INFO,POLICE_DIVISION_1,WARD_1,WARD_3,WARD_2,WARD_4,NAME,DISTRICT,RLC,MAIN
0,4678,13465959.0,Richmond St E,Parliament St,1,315768.179,4834966.252,-79.364023,43.654559,1,...,NaN,51,Toronto Centre(13),NaN,NaN,NaN,Richmond St E And Parliament St,Toronto and East York,6001,RICHMOND ST E
1,4679,13467993.0,Lake Shore Blvd W,York St,2,314411.515,4833508.939,-79.380868,43.641460,2,...,WB LAKE SHORE BLVD,52,Spadina-Fort York(10),NaN,NaN,NaN,Lake Shore Blvd W And York St,Toronto and East York,6002,LAKE SHORE BLVD W
2,4680,13444656.0,Steeles Ave W,Carpenter Rd,3,309017.922,4850229.154,-79.447589,43.792013,3,...,NaN,32,York Centre(6),NaN,NaN,NaN,Steeles Ave W And Carpenter Rd / Private Acces...,North York,6003,STEELES AVE W
3,4681,13444138.0,Steeles Ave W,Hilda Ave,4,310492.057,4850673.973,-79.429267,43.796007,4,...,NaN,32,Willowdale(18),NaN,NaN,NaN,Steeles Ave W And Hilda Ave,North York,6004,STEELES AVE W
4,4682,13451893.0,Albion Rd,Silverstone Dr,5,296738.069,4844782.178,-79.600094,43.742952,5,...,NaN,23,Etobicoke North(1),NaN,NaN,NaN,Albion Rd And Silverstone Dr,Etobicoke York,6005,ALBION RD


In [7]:
# Create a set of starbucks locations lat and lng combinations
lng_lats = []

lngs = lights_df['LONGITUDE']
lats = lights_df['LATITUDE']

lng_lats = zip(lngs, lats)

neighbourhood_id = []
neighbourhood_name = []
lats_list = []
lngs_list = []

for lng_lat in lng_lats:
    point = Point(lng_lat[0], lng_lat[1])
    
    for i in np.arange(len(regions)):
        poly = regions.loc[i, 'geometry']
        
        if point.within(poly):
            neighbourhood_id.append(regions.loc[i, 'FIELD_6'])
            neighbourhood_name.append(regions.loc[i, 'FIELD_7'])
            lats_list.append(lng_lat[1])
            lngs_list.append(lng_lat[0])

In [8]:
lights_dict = {
    'Lat' : lats_list,
    'Long' : lngs_list,
    'Hood ID' : neighbourhood_id,
    'Neighbourhood' : neighbourhood_name
}

clean_lights = pd.DataFrame(lights_dict)

clean_lights

,Lat,Long,Hood ID,Neighbourhood
0,43.654559,-79.364023,73,Moss Park (73)
1,43.641460,-79.380868,77,Waterfront Communities-The Island (77)
2,43.792013,-79.447589,35,Westminster-Branson (35)
3,43.796007,-79.429267,36,Newtonbrook West (36)
4,43.742952,-79.600094,1,West Humber-Clairville (1)
...,...,...,...,...
143,43.787283,-79.470089,34,Bathurst Manor (34)
144,43.668464,-79.439035,93,Dovercourt-Wallace Emerson-Junction (93)
145,43.644648,-79.523438,14,Islington-City Centre West (14)
146,43.755454,-79.614090,1,West Humber-Clairville (1)


In [9]:
lights_val_counts = clean_lights['Neighbourhood'].value_counts()

In [10]:
# number of red lights each location
lights_val_counts = lights_val_counts.reset_index()
lights_val_counts.columns = ['Neighbourhood', 'Number of Red Lights']

lights_val_counts

,Neighbourhood,Number of Red Lights
0,Islington-City Centre West (14),7
1,Woburn (137),6
2,Steeles (116),6
3,West Humber-Clairville (1),6
4,Waterfront Communities-The Island (77),6
...,...,...
72,Highland Creek (134),1
73,Keelesdale-Eglinton West (110),1
74,Lawrence Park South (103),1
75,Forest Hill South (101),1


In [11]:
clean_lights_df = clean_lights.drop_duplicates(subset='Neighbourhood', keep='first')
clean_lights_df

,Lat,Long,Hood ID,Neighbourhood
0,43.654559,-79.364023,73,Moss Park (73)
1,43.641460,-79.380868,77,Waterfront Communities-The Island (77)
2,43.792013,-79.447589,35,Westminster-Branson (35)
3,43.796007,-79.429267,36,Newtonbrook West (36)
4,43.742952,-79.600094,1,West Humber-Clairville (1)
...,...,...,...,...
131,43.715456,-79.507224,28,Rustic (28)
132,43.722255,-79.415609,39,Bedford Park-Nortown (39)
137,43.801974,-79.199771,131,Rouge (131)
141,43.654707,-79.460048,87,High Park-Swansea (87)


In [12]:

lightslights_final = pd.merge(lights_val_counts, clean_lights_df, on='Neighbourhood', how='inner')_final.head()

,Neighbourhood,Number of Red Lights,Lat,Long,Hood ID
0,Islington-City Centre West (14),7,43.612307,-79.562677,14
1,Woburn (137),6,43.757721,-79.235267,137
2,Steeles (116),6,43.821392,-79.316925,116
3,West Humber-Clairville (1),6,43.742952,-79.600094,1
4,Waterfront Communities-The Island (77),6,43.641460,-79.380868,77


In [13]:
lights_final.drop(['Lat', 'Long'], axis='columns', inplace=True)
lights_final = lights_final[['Hood ID','Neighbourhood','Number of Red Lights']]
lights_final.rename(columns={"Hood ID": "Hood_ID"}, inplace=True)

In [14]:
lights_final

,Hood_ID,Neighbourhood,Number of Red Lights
0,14,Islington-City Centre West (14),7
1,137,Woburn (137),6
2,116,Steeles (116),6
3,1,West Humber-Clairville (1),6
4,77,Waterfront Communities-The Island (77),6
...,...,...,...
72,134,Highland Creek (134),1
73,110,Keelesdale-Eglinton West (110),1
74,103,Lawrence Park South (103),1
75,101,Forest Hill South (101),1


In [15]:
lights_final.to_csv('output_data/lights_final.csv')